# House Price Modeling: Linear & Multiple Regression
This notebook loads the provided housing dataset and performs the requested analyses:
1. Simple linear regression: **Price ~ Size**
2. Simple linear regression: **Price ~ Sqft Living**
3. Multiple linear regression: **Price ~ Sqft Living + Bedrooms + Age + Waterfront**
4. Multiple linear regression: **Price ~ Size + Bedrooms + Age + Distance_to_city_center**
5. Model evaluation with **R-squared**
6. Assumption checks (residual plots, Q-Q plot) and discussion

> **Note:** The code auto-detects columns with flexible names (e.g., `sqft_living` vs `size`) and derives `age` from `yr_built` and `date_sold` if present.

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)


## Load Dataset

In [8]:
import os
import pandas as pd

# --- find the data file ---
data_path = None
for cand in ["dataset/housing_2.csv", "dataset/housing.csv",
             "dataset/housing_2.xlsx", "dataset/housing.xlsx"]:
    if os.path.exists(cand):
        data_path = cand
        break

if data_path is None:
    raise FileNotFoundError("Could not find dataset/housing_2 or dataset/housing (csv/xlsx).")

# --- robust loader: try multiple encodings / formats ---
def robust_read(path):
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    # try CSV with several encodings and parsers
    encodings = ["utf-8", "utf-8-sig", "latin1", "cp1252"]
    for enc in encodings:
        try:
            # engine='python' + sep=None lets pandas sniff odd delimiters
            return pd.read_csv(path, encoding=enc, engine="python", sep=None, on_bad_lines="skip")
        except UnicodeDecodeError:
            continue
        except Exception as e:
            # try again with default engine in case dialect is clean
            try:
                return pd.read_csv(path, encoding=enc, on_bad_lines="skip")
            except Exception:
                last_err = e
                continue
    # final attempt: replace undecodable bytes
    return pd.read_csv(path, encoding="latin1", engine="python", sep=None,
                       on_bad_lines="skip", encoding_errors="replace")

df_raw = robust_read(data_path)

# --- standardize column names ---
df = df_raw.copy()
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(r"[^a-z0-9]+", "_", regex=True)
              .str.strip("_")
)

print(f"Loaded: {data_path}  |  shape: {df.shape}")
display(df.head())


Loaded: dataset/housing_2.csv  |  shape: (36, 2)


,columns,keep
0,Date:ÊThe date when the property was sold. Thi...,Yes
1,the temporal trends in property prices.,NaN
2,Price:The sale price of the property in USD. T...,Yes
3,to predict.,NaN
4,Bedrooms:The number of bedrooms in the propert...,Yes


## Prepare and Harmonize Features
We will map available columns to the required names, and derive **age** when possible from `yr_built` and `date_sold` (or `sale_date`).

In [13]:
# --- Imports ---
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

pd.set_option('display.max_columns', None)

# --- 0) Locate & Load the dataset (robust to path + encoding) ---
# Try common locations relative to where your notebook lives or project root
candidate_paths = [
    "dataset/housing_2.csv", "dataset/housing.csv",
    "AIT110/topic1/dataset/housing_2.csv", "AIT110/topic1/dataset/housing.csv",
    "/mnt/data/housing_2.csv", "/mnt/data/housing.csv",
    "dataset/housing_2.xlsx", "dataset/housing.xlsx",
    "AIT110/topic1/dataset/housing_2.xlsx", "AIT110/topic1/dataset/housing.xlsx",
]

data_path = next((p for p in candidate_paths if os.path.exists(p)), None)
if data_path is None:
    raise FileNotFoundError("Could not find housing file in expected locations.\n"
                            "Checked:\n" + "\n".join(candidate_paths))

def robust_read(path):
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    # Try multiple encodings for CSV
    for enc in ["utf-8", "utf-8-sig", "cp1252", "latin1"]:
        try:
            return pd.read_csv(path, encoding=enc, engine="python", sep=None, on_bad_lines="skip")
        except UnicodeDecodeError:
            continue
        except Exception:
            # try default engine as a second attempt
            try:
                return pd.read_csv(path, encoding=enc, on_bad_lines="skip")
            except Exception:
                continue
    # Last resort: replace undecodable bytes
    return pd.read_csv(path, encoding="latin1", engine="python", sep=None,
                       on_bad_lines="skip", encoding_errors="replace")

df_raw = robust_read(data_path)

# --- 1) Standardize column names ---
df = df_raw.copy()
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(r"[^a-z0-9]+", "_", regex=True)
              .str.strip("_")
)

print(f"Loaded: {data_path} | shape={df.shape}")
print("Columns:", list(df.columns))

# --- 2) Flexible column mapping (your pipeline, upgraded) ---
def pick_col(candidates, contains=None):
    """Return the first exact match; if none, return the first column whose
    name contains any of the 'contains' tokens (case-insensitive)."""
    for c in candidates:
        if c in df.columns:
            return c
    if contains:
        tokens = [t.lower() for t in contains]
        for col in df.columns:
            name = col.lower()
            if any(t in name for t in tokens):
                return col
    return None

# Identify columns
price_col = pick_col(
    ['price', 'sale_price', 'log_price', 'saleprice', 'price_usd', 'median_house_value'],
    contains=['price','value']
)
size_col  = pick_col(
    ['size','sqft','sqft_size','sqft_living','living_area','sqftliving','gr_liv_area','total_sqft','square_feet'],
    contains=['sqft','area','size']
)
sqft_col  = pick_col(
    ['sqft_living','sqftliving','living_area','size','sqft','gr_liv_area','total_sqft','square_feet'],
    contains=['sqft','living','area']
)
beds_col  = pick_col(
    ['bedrooms','beds','bedroom','num_bedrooms','br'],
    contains=['bed']
)
age_col   = pick_col(
    ['age','age_auto','years_old'],
    contains=['age']
)
dist_col  = pick_col(
    ['distance_to_city_center','distance','dist_city_center','dist_to_city_center','distance_from_city','city_distance'],
    contains=['dist','city']
)
water_col = pick_col(
    ['waterfront','is_waterfront','on_water','water_front'],
    contains=['water']
)

# Derive age if necessary from yr_built & sale date
yr_built_col = pick_col(['yr_built','year_built','built_year'], contains=['built'])
date_sold_col = pick_col(['date_sold','sale_date','sold_date','date'], contains=['date','sold'])

def to_datetime_safe(s):
    try:
        return pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
    except Exception:
        return pd.to_datetime(s, errors='coerce')

if age_col is None:
    if yr_built_col is not None and date_sold_col is not None:
        ds = to_datetime_safe(df[date_sold_col])
        ref_year = np.where(ds.notna(), ds.dt.year, datetime.now().year)
        df['age_auto'] = (ref_year - pd.to_numeric(df[yr_built_col], errors='coerce')).astype(float)
        age_col = 'age_auto'
    elif yr_built_col is not None:
        df['age_auto'] = (datetime.now().year - pd.to_numeric(df[yr_built_col], errors='coerce')).astype(float)
        age_col = 'age_auto'

# Ensure Waterfront becomes 0/1 (create if missing)
if water_col is not None:
    w = df[water_col].copy()
    if w.dtype == 'O':
        ws = w.astype(str).str.lower().str.strip()
        truthy = {'1','true','yes','y','t'}
        w = ws.apply(lambda x: 1.0 if x in truthy else (0.0 if x.replace('.','',1).isdigit() and float(x)==0 else np.nan))
        w = w.fillna(0.0).astype(float)
    else:
        w = pd.to_numeric(w, errors='coerce').fillna(0.0)
        if not set(np.unique(w.dropna())).issubset({0.0,1.0}):
            w = (w > 0).astype(float)
    df['_waterfront_bin'] = w.astype(float)
    water_col = '_waterfront_bin'
else:
    df['_waterfront_bin'] = 0.0
    water_col = '_waterfront_bin'

print("Mapped columns:", {
    "price": price_col, "size": size_col, "sqft_living": sqft_col,
    "bedrooms": beds_col, "age": age_col,
    "distance_to_city_center": dist_col, "waterfront": water_col
})

# --- 3) Build modeling frame (your pipeline continues) ---
if price_col is None:
    raise ValueError("A price column was not found. Please set price_col manually after checking printed columns.")

df_model = df.copy()
if price_col == 'log_price':
    df_model['price'] = np.exp(df_model['log_price'])
else:
    df_model['price'] = pd.to_numeric(df_model[price_col], errors='coerce')

# Coerce numerics for predictors that exist
for c in [size_col, sqft_col, beds_col, age_col, dist_col, water_col]:
    if c is not None:
        df_model[c] = pd.to_numeric(df_model[c], errors='coerce')

# Keep rows with price present
df_model = df_model[df_model['price'].notna()].copy()
display(df_model.head())

# --- 4) Helper: OLS with intercept & pretty equation ---
def fit_ols(y, X_df):
    X = sm.add_constant(X_df, has_constant='add')
    return sm.OLS(y, X, missing='drop').fit()

def regression_equation(model, y_name='Price'):
    params = model.params
    intercept = params.get('const', 0.0)
    terms = [f"{coef:.4f}*{name}" for name, coef in params.items() if name != 'const']
    rhs = f"{intercept:.4f}" + ("" if not terms else " + " + " + ".join(terms))
    return f"{y_name} = {rhs}"

# --- 5) Model 1: Price ~ Size ---
size_like = size_col if size_col is not None else sqft_col
if size_like is None:
    raise ValueError("No 'Size' or 'Sqft Living' style column found for Model 1.")
m1 = fit_ols(df_model['price'], df_model[[size_like]])
print("\n[Model 1] Price ~ Size-like")
print(regression_equation(m1, 'Price'))
print("R-squared:", round(m1.rsquared, 4))

# --- 6) Model 2: Price ~ Sqft Living ---
sqft_like = sqft_col if sqft_col is not None else size_col
if sqft_like is None:
    raise ValueError("No 'Sqft Living' or 'Size' style column found for Model 2.")
m2 = fit_ols(df_model['price'], df_model[[sqft_like]])
print("\n[Model 2] Price ~ Sqft Living-like")
print(regression_equation(m2, 'Price'))
print("R-squared:", round(m2.rsquared, 4))

# --- 7) Model 3: Price ~ SqftLiving + Bedrooms + Age + Waterfront ---
features_m3 = [c for c in [sqft_col, beds_col, age_col, water_col] if c is not None]
if len(features_m3) == 0:
    raise ValueError("No predictors available for Model 3.")
m3 = fit_ols(df_model['price'], df_model[features_m3])
print("\n[Model 3] Price ~", " + ".join(features_m3))
print(regression_equation(m3, 'Price'))
print("R-squared:", round(m3.rsquared, 4))

# Strongest impact via standardized coefficients
X_m3 = df_model[features_m3].copy().dropna()
y_m3 = df_model.loc[X_m3.index, 'price']
X_std = (X_m3 - X_m3.mean())/X_m3.std(ddof=0)
m3_std = fit_ols(y_m3, X_std)
coef_abs = m3_std.params.drop('const').abs().sort_values(ascending=False)
print("Approx strongest impact (|standardized coef|):")
print(coef_abs.head())

# --- 8) Model 4: Price ~ Size + Bedrooms + Age + Distance_to_city_center ---
features_m4 = [c for c in [size_col, beds_col, age_col, dist_col] if c is not None]
if len(features_m4) == 0:
    raise ValueError("No predictors available for Model 4.")
m4 = fit_ols(df_model['price'], df_model[features_m4])
print("\n[Model 4] Price ~", " + ".join(features_m4))
print(regression_equation(m4, 'Price'))
print("R-squared:", round(m4.rsquared, 4))

X_m4 = df_model[features_m4].copy().dropna()
y_m4 = df_model.loc[X_m4.index, 'price']
X_std4 = (X_m4 - X_m4.mean())/X_m4.std(ddof=0)
m4_std = fit_ols(y_m4, X_std4)
coef_abs4 = m4_std.params.drop('const').abs().sort_values(ascending=False)
print("Approx strongest impact (|standardized coef|):")
print(coef_abs4.head())

# --- 9) Compare R^2 across models ---
eval_df = pd.DataFrame([
    {"model": f"Price ~ {size_like}", "r_squared": round(m1.rsquared, 4)},
    {"model": f"Price ~ {sqft_like}", "r_squared": round(m2.rsquared, 4)},
    {"model": f"Price ~ {' + '.join(features_m3)}", "r_squared": round(m3.rsquared, 4)},
    {"model": f"Price ~ {' + '.join(features_m4)}", "r_squared": round(m4.rsquared, 4)},
])
display(eval_df)

# --- 10) Residual diagnostics (use Model 4; fallback to Model 3) ---
model_diag = m4 if 'm4' in globals() else m3
fitted = model_diag.fittedvalues
resid = model_diag.resid

plt.figure()
plt.scatter(fitted, resid, alpha=0.6)
plt.axhline(0, linestyle='--')
plt.title('Residuals vs Fitted')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.show()

fig = sm.qqplot(resid, line='45')
plt.title('Q-Q Plot of Residuals')
plt.show()

plt.figure()
plt.scatter(fitted, np.sqrt(np.abs(resid)), alpha=0.6)
plt.title('Scale-Location (sqrt|Residuals| vs Fitted)')
plt.xlabel('Fitted values')
plt.ylabel('sqrt(|Residuals|)')
plt.show()

print("\nAssumption guide:")
print("- Linearity: Residuals should randomly scatter around 0.")
print("- Homoscedasticity: Roughly constant spread of residuals.")
print("- Normality: Q-Q close to 45° line. If not, consider transforms (e.g., log(Price)).")


Loaded: dataset/housing_2.csv | shape=(36, 2)
Columns: ['columns', 'keep']
Mapped columns: {'price': None, 'size': None, 'sqft_living': None, 'bedrooms': None, 'age': None, 'distance_to_city_center': None, 'waterfront': '_waterfront_bin'}


ValueError: A price column was not found. Please set price_col manually after checking printed columns.

In [ ]:
def fit_ols(y, X_df):
    X = sm.add_constant(X_df, has_constant='add')
    model = sm.OLS(y, X, missing='drop').fit()
    return model

def regression_equation(model, y_name='Price'):
    params = model.params
    terms = []
    intercept = params.get('const', 0.0)
    for name, coef in params.items():
        if name == 'const':
            continue
        terms.append(f"{coef:.4f}*{name}")
    rhs = f"{intercept:.4f} + " + " + ".join(terms) if terms else f"{intercept:.4f}"
    return f"{y_name} = {rhs}"


## 1) Simple Linear Regression — Price ~ Size

In [ ]:
# Choose a size-like feature (fall back to sqft_col if needed)
size_like = size_col if size_col is not None else sqft_col
if size_like is None:
    raise ValueError("No 'Size' or 'Sqft Living' column found for the simple model.")

m1 = fit_ols(df_model['price'], df_model[[size_like]])
print("Regression equation:")
print(regression_equation(m1, 'Price'))
print("\nCoefficients:\n", m1.params)
print("\nR-squared:", round(m1.rsquared, 4))


**Interpretation (automated):** The slope is the change in predicted **Price** for a one-unit increase in **Size** (square feet). The intercept is the predicted **Price** when **Size = 0** (a baseline, often outside realistic range).

## 2) Simple Linear Regression — Price ~ Sqft Living

In [ ]:
# Choose a sqft_living-like feature (fall back to size_col if needed)
sqft_like = sqft_col if sqft_col is not None else size_col
if sqft_like is None:
    raise ValueError("No 'Sqft Living' or 'Size' column found for this model.")

m2 = fit_ols(df_model['price'], df_model[[sqft_like]])
print("Regression equation:")
print(regression_equation(m2, 'Price'))
print("\nCoefficients:\n", m2.params)
print("\nR-squared:", round(m2.rsquared, 4))


**Interpretation (automated):** The slope is the change in predicted **Price** per additional square foot of **Sqft Living**. The intercept is the baseline **Price** at 0 square feet.

## 3) Multiple Linear Regression — Price ~ Sqft Living + Bedrooms + Age + Waterfront

In [ ]:
# Build feature list for model 3
features_m3 = []
names_m3 = {'sqft_living': sqft_col, 'bedrooms': beds_col, 'age': age_col, 'waterfront': water_col}

for k, c in names_m3.items():
    if c is not None and c in df_model.columns:
        features_m3.append(c)

if len(features_m3) == 0:
    raise ValueError("No predictors available for Model 3.")

m3 = fit_ols(df_model['price'], df_model[features_m3])
print("Regression equation:")
print(regression_equation(m3, 'Price'))
print("\nCoefficients:\n", m3.params)
print("\nR-squared:", round(m3.rsquared, 4))

# Identify strongest impact by absolute standardized coefficient (simple proxy)
X_m3 = df_model[features_m3].copy().dropna()
y_m3 = df_model.loc[X_m3.index, 'price']
X_std = (X_m3 - X_m3.mean())/X_m3.std(ddof=0)
m3_std = fit_ols(y_m3, X_std)
coef_abs = m3_std.params.drop('const').abs().sort_values(ascending=False)
print("\nApprox. strongest impact (by |standardized coef|):\n", coef_abs.head())


> **Interpretation Tip:** Positive coefficients increase price; negative coefficients decrease price. The variable with the largest absolute standardized coefficient generally has the strongest impact (holding other variables constant).

## 4) Multiple Linear Regression — Price ~ Size + Bedrooms + Age + Distance_to_city_center

In [ ]:
# Build feature list for model 4
features_m4 = []
names_m4 = {'size': size_col, 'bedrooms': beds_col, 'age': age_col, 'distance_to_city_center': dist_col}

for k, c in names_m4.items():
    if c is not None and c in df_model.columns:
        features_m4.append(c)

if len(features_m4) == 0:
    raise ValueError("No predictors available for Model 4.")

m4 = fit_ols(df_model['price'], df_model[features_m4])
print("Regression equation:")
print(regression_equation(m4, 'Price'))
print("\nCoefficients:\n", m4.params)
print("\nR-squared:", round(m4.rsquared, 4))

# Strongest impact via standardized coefficients
X_m4 = df_model[features_m4].copy().dropna()
y_m4 = df_model.loc[X_m4.index, 'price']
X_std4 = (X_m4 - X_m4.mean())/X_m4.std(ddof=0)
m4_std = fit_ols(y_m4, X_std4)
coef_abs4 = m4_std.params.drop('const').abs().sort_values(ascending=False)
print("\nApprox. strongest impact (by |standardized coef|):\n", coef_abs4.head())


## 5) Model Evaluation — R-squared and Comparison

In [ ]:
results = []
results.append({"model": f"Price ~ {size_like}", "r_squared": round(m1.rsquared,4)})
results.append({"model": f"Price ~ {sqft_like}", "r_squared": round(m2.rsquared,4)})
results.append({"model": f"Price ~ {' + '.join(features_m3)}", "r_squared": round(m3.rsquared,4)})
results.append({"model": f"Price ~ {' + '.join(features_m4)}", "r_squared": round(m4.rsquared,4)})
eval_df = pd.DataFrame(results)
eval_df


**Interpretation:** Higher **R-squared** indicates a better fit (more variance in **Price** explained). Multiple regression models typically have higher R-squared than simple models because they use more information.

## 6) Assumption Checks — Residual Diagnostics (using Model 4 by default)
We visualize residuals vs. fitted values and a Q-Q plot to assess normality.

In [ ]:
# Choose model for diagnostics (prefer model 4; if not available, fallback to model 3)
model_diag = m4 if 'm4' in globals() else m3

fitted = model_diag.fittedvalues
resid = model_diag.resid

# Residuals vs Fitted
plt.figure()
plt.scatter(fitted, resid, alpha=0.6)
plt.axhline(0, linestyle='--')
plt.title('Residuals vs Fitted')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.show()

# Q-Q plot for residuals
fig = sm.qqplot(resid, line='45')
plt.title('Q-Q Plot of Residuals')
plt.show()

# Scale-Location (Spread vs Fitted) — optional but helpful
plt.figure()
plt.scatter(fitted, np.sqrt(np.abs(resid)), alpha=0.6)
plt.title('Scale-Location (sqrt|Residuals| vs Fitted)')
plt.xlabel('Fitted values')
plt.ylabel('sqrt(|Residuals|)')
plt.show()


### Discussion of Assumptions
- **Linearity:** Residuals should scatter randomly around zero (no clear curves). If curved patterns appear, consider adding polynomial terms or transforming predictors.
- **Homoscedasticity (constant variance):** The spread of residuals should be roughly constant across fitted values. If you see a fan-shaped pattern, try transforming **Price** (e.g., log) or using weighted least squares.
- **Normality of residuals:** Q-Q plot points should lie close to the 45° line. Strong deviations suggest non-normality; consider transforming **Price** or using robust methods.
- **Outliers/Influence:** Investigate extreme residuals or high-leverage points (e.g., Cook’s distance) and validate data quality.